In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *


#clasificadores
from utils.clasificacion import *

#adaptacion
from utils.adaptacion import *

#otros
import os
import numpy as np
import pandas as pd

#variables para guardar los resultados
tipo = pruebas[0]
dataset_name = datasets[0]
dims = dimensions[dataset_name]

Using Theano backend.
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
print tipo
print dataset_name
print dims
print data_path

baseline
amazon
3000
data


## Pruebas con el dataset Amazon

In [4]:
# cargando dataset Amazon
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
unlabeled = dataset_object.unlabeled
domains = dataset_object.domains

Dataset already splitted


## Baseline

In [ ]:
%%time

i = 1
for tgt in domains:
    print "Entrenando dominio %d de %d" % (i, len(domains))
    X_tr = np.asarray(labeled[tgt]['X_tr'].todense())
    y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)

    get_best_score(X_tr, y_tr, clasifier='SVC', n_jobs=1)
    
    i = i+1
    
print "\nTarea terminada."

In [ ]:
# se obtienen todos los valores X disponibles para realizar adaptacion
X = dataset_object.get_all_X()
X = np.asarray(X)

print "Todos los datos disponibles obtenidos"

In [ ]:
%%time

i = 1
for tgt in domains:
    print "Entrenando dominio %d de %d" % (i, len(domains))
    X_tr = np.asarray(labeled[tgt]['X_tr'].todense())
    y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)

    get_best_score(X_tr, y_tr, clasifier='SVC', n_jobs=4)
    
    i = i+1
    
print "\nTarea terminada."

## GFK

In [ ]:
size_dims = [int(dims/4), int(dims/2)]
n_subs = [10, 50, 100]

parameters = {
    'dims': size_dims,
    'n_subs': n_subs
}

In [ ]:
%%time

tgt = domains[0]

X_tr = np.asarray(labeled[tgt]['X_tr'].todense())
y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)

X_ts = np.asarray(labeled[tgt]['X_ts'].todense())
y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

gfk, score = gfk_grid_search(parameters, X_tr, y_tr, X_ts, n_jobs=1)

In [ ]:
%%time

tgt = domains[0]

X_tr = np.asarray(labeled[tgt]['X_tr'].todense())
y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)

X_ts = np.asarray(labeled[tgt]['X_ts'].todense())
y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

gfk, score = gfk_grid_search(parameters, X_tr, y_tr, X_ts, n_jobs=4)

# mSDA

In [ ]:
from mSDA import msda

In [ ]:
noises = [0.3, 0.5, 0.8]
layers_sizes = [1, 3, 5]

parametros = {
    'noises': noises,
    'layers': layers_sizes
}

sub_size = 10000

In [ ]:
# 10000 ejemplos - 2000 dimensiones
Xs_1 = np.random.randint(0, 2, size = (10000, 2000))
Xs_1.shape

In [ ]:
Xs_1[0]

In [ ]:
%%time
t_adaptar, train_mappings = adapt_msda(Xs_1.transpose(), pr=0.3, n_layers=5)

In [ ]:
# 10000 ejemplos - 5000 dimensiones
Xs_1 = np.random.randint(0, 2, size = (10000, 5000))
Xs_1.shape

In [ ]:
Xs_1[0]

In [ ]:
%%time
t_adaptar, train_mappings = adapt_msda(Xs_1.transpose(), pr=0.3, n_layers=3)
t_adaptar

# mSDA Theano

In [19]:
from mSDA.msda_theano import mSDATheano
import theano.tensor as T
from sklearn.externals import joblib

In [12]:
# 10000 ejemplos - 2000 dimensiones
Xs_1 = np.random.randint(0, 2, size = (10000, 3000))
Xs_1.shape

(10000, 3000)

In [16]:
x = T.dmatrix('x')
msda_adapter = mSDATheano(x, l=3, pr=0.3)

In [17]:
msda_adapter.fit(Xs_1)

Entrenando capa  1
Entrenando capa  2
Entrenando capa  3
Entrenado en 7.51m



7.5092024326324465

In [20]:
joblib.dump(msda_adapter, "test_msda.pkl")

['test_msda.pkl']

In [21]:
new_msda = joblib.load("test_msda.pkl")

In [23]:
new_msda.predict(Xs_1[:1000, :]).shape

(1000, 3000)